<a href="https://colab.research.google.com/github/rish4/NewColonoscopy/blob/main/New_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
import keras
import tensorflow as tf
from keras import backend as K
from keras import metrics
from keras.regularizers import l2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, InputLayer, Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.metrics import AUC
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix
from keras import models
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import numpy as np
from keras.preprocessing import image
from IPython.display import display
from PIL import Image
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
import numpy

In [2]:
## Set file paths to image files

train_path = "/content/drive/MyDrive/NewDB/Training set"
test_path = "/content/drive/MyDrive/NewDB/Test set"

## Set up hyperparameters that will be used later
hyper_dimension = 64
hyper_batch_size = 32
hyper_epochs = 100
hyper_channels = 3
hyper_mode = 'rgb'



In [3]:

'''cnn_model = cnn.fit_generator(train_generator, 
                              steps_per_epoch = len(train_generator), 
                              epochs = 20, 
                              validation_data = test_generator,
                              validation_steps = len(test_generator)
                              )'''

'cnn_model = cnn.fit_generator(train_generator, \n                              steps_per_epoch = len(train_generator), \n                              epochs = 20, \n                              validation_data = test_generator,\n                              validation_steps = len(test_generator)\n                              )'

**For checking best epochs and batch size**

In [14]:
def create_model():
  cnn=Sequential()
  cnn.add(InputLayer(input_shape=(hyper_dimension, hyper_dimension, hyper_channels)))

  cnn.add(Conv2D(filters=16, kernel_size=3, activation='relu'))
  cnn.add(MaxPooling2D(pool_size=(2,2)))

  cnn.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
  cnn.add(MaxPooling2D(pool_size=(2,2)))

  cnn.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
  cnn.add(MaxPooling2D(pool_size=(2,2)))

  cnn.add(Flatten())

  cnn.add(Dense(activation='relu', units=128))
  cnn.add(Dense(activation='sigmoid', units=1))

  #cnn.summary()

  cnn.compile(optimizer= 'adam', loss='binary_crossentropy', metrics=['accuracy'])
  return cnn

seed = 7
numpy.random.seed(seed)

train_datagen = ImageDataGenerator(rescale=1.0/255.0, 
                                   shear_range = 0.2,
                                   zoom_range = 0.2, 
                                   horizontal_flip = True )

#val_datagen = ImageDataGenerator(rescale=1.0/255.0) 
test_datagen = ImageDataGenerator(rescale=1.0/255.0) 

train_generator = train_datagen.flow_from_directory(directory = train_path, 
                                                    target_size = (hyper_dimension, hyper_dimension),
                                                    batch_size = hyper_batch_size, 
                                                    color_mode = hyper_mode,
                                                    class_mode = 'binary', 
                                                    seed = 42)

test_generator = test_datagen.flow_from_directory(directory = test_path, 
                                                 target_size = (hyper_dimension, hyper_dimension),
                                                 batch_size = hyper_batch_size, 
                                                 class_mode = 'binary',
                                                 color_mode = hyper_mode,
                                                 shuffle=False,
                                                 seed = 42)

test_generator.reset()

#datagen = ImageDataGenerator(rescale=1.0/255.0)
#dataset = datagen.flow_from_directory('dataset_dog_breeds/train/', class_mode='categorical')

X, Y = train_generator.next()

cnn = KerasClassifier(build_fn=create_model, verbose=0)

batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']

'''param_grid = {
    'batch_size': batch_size, 
    'epochs':epochs,
    'optimizer': optimizer,
    'init_mode': init_mode,
    'activation': activation
}'''

param_grid= dict(epochs=epochs, batch_size=batch_size)
grid = GridSearchCV(estimator=cnn, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Found 1656 images belonging to 2 classes.
Found 415 images belonging to 2 classes.
Best: 0.787879 using {'batch_size': 40, 'epochs': 100}
0.533333 (0.120909) with: {'batch_size': 10, 'epochs': 10}
0.593939 (0.111423) with: {'batch_size': 10, 'epochs': 50}
0.627273 (0.122643) with: {'batch_size': 10, 'epochs': 100}
0.463636 (0.115708) with: {'batch_size': 20, 'epochs': 10}
0.627273 (0.122643) with: {'batch_size': 20, 'epochs': 50}
0.627273 (0.122643) with: {'batch_size': 20, 'epochs': 100}
0.566667 (0.101323) with: {'batch_size': 40, 'epochs': 10}
0.751515 (0.081762) with: {'batch_size': 40, 'epochs': 50}
0.787879 (0.154516) with: {'batch_size': 40, 'epochs': 100}
0.466667 (0.059997) with: {'batch_size': 60, 'epochs': 10}
0.724242 (0.193608) with: {'batch_size': 60, 'epochs': 50}
0.727273 (0.222681) with: {'batch_size': 60, 'epochs': 100}
0.657576 (0.029998) with: {'batch_size': 80, 'epochs': 10}
0.660606 (0.148701) with: {'batch_size': 80, 'epochs': 50}
0.693939 (0.183377) with: {'batc

**For checking best optimizer**

In [15]:
def create_model(optimizer='adam'):
  cnn=Sequential()
  cnn.add(InputLayer(input_shape=(hyper_dimension, hyper_dimension, hyper_channels)))

  cnn.add(Conv2D(filters=16, kernel_size=3, activation='relu'))
  cnn.add(MaxPooling2D(pool_size=(2,2)))

  cnn.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
  cnn.add(MaxPooling2D(pool_size=(2,2)))

  cnn.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
  cnn.add(MaxPooling2D(pool_size=(2,2)))

  cnn.add(Flatten())

  cnn.add(Dense(activation='relu', units=128))
  cnn.add(Dense(activation='sigmoid', units=1))

  #cnn.summary()

  cnn.compile(optimizer= 'adam', loss='binary_crossentropy', metrics=['accuracy'])
  return cnn

seed = 7
numpy.random.seed(seed)

train_datagen = ImageDataGenerator(rescale=1.0/255.0, 
                                   shear_range = 0.2,
                                   zoom_range = 0.2, 
                                   horizontal_flip = True )

#val_datagen = ImageDataGenerator(rescale=1.0/255.0) 
test_datagen = ImageDataGenerator(rescale=1.0/255.0) 

train_generator = train_datagen.flow_from_directory(directory = train_path, 
                                                    target_size = (hyper_dimension, hyper_dimension),
                                                    batch_size = hyper_batch_size, 
                                                    color_mode = hyper_mode,
                                                    class_mode = 'binary', 
                                                    seed = 42)

test_generator = test_datagen.flow_from_directory(directory = test_path, 
                                                 target_size = (hyper_dimension, hyper_dimension),
                                                 batch_size = hyper_batch_size, 
                                                 class_mode = 'binary',
                                                 color_mode = hyper_mode,
                                                 shuffle=False,
                                                 seed = 42)

test_generator.reset()

#datagen = ImageDataGenerator(rescale=1.0/255.0)
#dataset = datagen.flow_from_directory('dataset_dog_breeds/train/', class_mode='categorical')

X, Y = train_generator.next()

cnn = KerasClassifier(build_fn=create_model, verbose=0,batch_size=40, epochs=100)

#batch_size = [10, 20, 40, 60, 80, 100]
#epochs = [10, 50, 100]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']

'''param_grid = {
    'batch_size': batch_size, 
    'epochs':epochs,
    'optimizer': optimizer,
    'init_mode': init_mode,
    'activation': activation
}'''

param_grid= dict(optimizer=optimizer)
grid = GridSearchCV(estimator=cnn, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Found 1656 images belonging to 2 classes.
Found 415 images belonging to 2 classes.
Best: 0.757576 using {'optimizer': 'Adagrad'}
0.660606 (0.148701) with: {'optimizer': 'SGD'}
0.727273 (0.222681) with: {'optimizer': 'RMSprop'}
0.757576 (0.186800) with: {'optimizer': 'Adagrad'}
0.724242 (0.144758) with: {'optimizer': 'Adadelta'}
0.693939 (0.183377) with: {'optimizer': 'Adam'}
0.690909 (0.107052) with: {'optimizer': 'Adamax'}
0.627273 (0.122643) with: {'optimizer': 'Nadam'}


**For checking best weight initializer**

In [17]:
def create_model(init_mode='uniform'):
  cnn=Sequential()
  cnn.add(InputLayer(input_shape=(hyper_dimension, hyper_dimension, hyper_channels)))

  cnn.add(Conv2D(filters=16, kernel_size=3, activation='relu'))
  cnn.add(MaxPooling2D(pool_size=(2,2)))

  cnn.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
  cnn.add(MaxPooling2D(pool_size=(2,2)))

  cnn.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
  cnn.add(MaxPooling2D(pool_size=(2,2)))

  cnn.add(Flatten())

  cnn.add(Dense(activation='relu', units=128))
  cnn.add(Dense(activation='sigmoid', units=1))

  #cnn.summary()

  cnn.compile(optimizer= 'adam', loss='binary_crossentropy', metrics=['accuracy'])
  return cnn

seed = 7
numpy.random.seed(seed)

train_datagen = ImageDataGenerator(rescale=1.0/255.0, 
                                   shear_range = 0.2,
                                   zoom_range = 0.2, 
                                   horizontal_flip = True )

#val_datagen = ImageDataGenerator(rescale=1.0/255.0) 
test_datagen = ImageDataGenerator(rescale=1.0/255.0) 

train_generator = train_datagen.flow_from_directory(directory = train_path, 
                                                    target_size = (hyper_dimension, hyper_dimension),
                                                    batch_size = hyper_batch_size, 
                                                    color_mode = hyper_mode,
                                                    class_mode = 'binary', 
                                                    seed = 42)

test_generator = test_datagen.flow_from_directory(directory = test_path, 
                                                 target_size = (hyper_dimension, hyper_dimension),
                                                 batch_size = hyper_batch_size, 
                                                 class_mode = 'binary',
                                                 color_mode = hyper_mode,
                                                 shuffle=False,
                                                 seed = 42)

test_generator.reset()

#datagen = ImageDataGenerator(rescale=1.0/255.0)
#dataset = datagen.flow_from_directory('dataset_dog_breeds/train/', class_mode='categorical')

X, Y = train_generator.next()

cnn = KerasClassifier(build_fn=create_model, verbose=0,batch_size=40, epochs=100)

#batch_size = [10, 20, 40, 60, 80, 100]
#epochs = [10, 50, 100]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']

'''param_grid = {
    'batch_size': batch_size, 
    'epochs':epochs,
    'optimizer': optimizer,
    'init_mode': init_mode,
    'activation': activation
}'''

param_grid= dict(init_mode=init_mode)
grid = GridSearchCV(estimator=cnn, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Found 1656 images belonging to 2 classes.
Found 415 images belonging to 2 classes.
Best: 0.757576 using {'init_mode': 'uniform'}
0.757576 (0.186800) with: {'init_mode': 'uniform'}
0.660606 (0.148701) with: {'init_mode': 'lecun_uniform'}
0.727273 (0.222681) with: {'init_mode': 'normal'}
0.624242 (0.149440) with: {'init_mode': 'zero'}
0.693939 (0.183377) with: {'init_mode': 'glorot_normal'}
0.693939 (0.150359) with: {'init_mode': 'glorot_uniform'}
0.657576 (0.080060) with: {'init_mode': 'he_normal'}
0.660606 (0.105322) with: {'init_mode': 'he_uniform'}


**For checking best activation function**

In [18]:
def create_model(activation='relu'):
  cnn=Sequential()
  cnn.add(InputLayer(input_shape=(hyper_dimension, hyper_dimension, hyper_channels)))

  cnn.add(Conv2D(filters=16, kernel_size=3, activation='relu'))
  cnn.add(MaxPooling2D(pool_size=(2,2)))

  cnn.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
  cnn.add(MaxPooling2D(pool_size=(2,2)))

  cnn.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
  cnn.add(MaxPooling2D(pool_size=(2,2)))

  cnn.add(Flatten())

  cnn.add(Dense(activation='relu', units=128))
  cnn.add(Dense(activation='sigmoid', units=1))

  #cnn.summary()

  cnn.compile(optimizer= 'adam', loss='binary_crossentropy', metrics=['accuracy'])
  return cnn

seed = 7
numpy.random.seed(seed)

train_datagen = ImageDataGenerator(rescale=1.0/255.0, 
                                   shear_range = 0.2,
                                   zoom_range = 0.2, 
                                   horizontal_flip = True )

#val_datagen = ImageDataGenerator(rescale=1.0/255.0) 
test_datagen = ImageDataGenerator(rescale=1.0/255.0) 

train_generator = train_datagen.flow_from_directory(directory = train_path, 
                                                    target_size = (hyper_dimension, hyper_dimension),
                                                    batch_size = hyper_batch_size, 
                                                    color_mode = hyper_mode,
                                                    class_mode = 'binary', 
                                                    seed = 42)

test_generator = test_datagen.flow_from_directory(directory = test_path, 
                                                 target_size = (hyper_dimension, hyper_dimension),
                                                 batch_size = hyper_batch_size, 
                                                 class_mode = 'binary',
                                                 color_mode = hyper_mode,
                                                 shuffle=False,
                                                 seed = 42)

test_generator.reset()

#datagen = ImageDataGenerator(rescale=1.0/255.0)
#dataset = datagen.flow_from_directory('dataset_dog_breeds/train/', class_mode='categorical')

X, Y = train_generator.next()

cnn = KerasClassifier(build_fn=create_model, verbose=0,batch_size=40, epochs=100)

#batch_size = [10, 20, 40, 60, 80, 100]
#epochs = [10, 50, 100]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']

'''param_grid = {
    'batch_size': batch_size, 
    'epochs':epochs,
    'optimizer': optimizer,
    'init_mode': init_mode,
    'activation': activation
}'''

param_grid= dict(activation=activation)
grid = GridSearchCV(estimator=cnn, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Found 1656 images belonging to 2 classes.
Found 415 images belonging to 2 classes.
Best: 0.757576 using {'activation': 'softsign'}
0.660606 (0.148701) with: {'activation': 'softmax'}
0.660606 (0.148701) with: {'activation': 'softplus'}
0.757576 (0.186800) with: {'activation': 'softsign'}
0.627273 (0.122643) with: {'activation': 'relu'}
0.593939 (0.111423) with: {'activation': 'tanh'}
0.693939 (0.183377) with: {'activation': 'sigmoid'}
0.693939 (0.183377) with: {'activation': 'hard_sigmoid'}
0.693939 (0.183377) with: {'activation': 'linear'}


**For checking best filters**

In [19]:
def create_model(filters=16):
  cnn=Sequential()
  cnn.add(InputLayer(input_shape=(hyper_dimension, hyper_dimension, hyper_channels)))

  cnn.add(Conv2D(filters=16, kernel_size=3, activation='relu'))
  cnn.add(MaxPooling2D(pool_size=(2,2)))

  cnn.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
  cnn.add(MaxPooling2D(pool_size=(2,2)))

  cnn.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
  cnn.add(MaxPooling2D(pool_size=(2,2)))

  cnn.add(Flatten())

  cnn.add(Dense(activation='relu', units=128))
  cnn.add(Dense(activation='sigmoid', units=1))

  #cnn.summary()

  cnn.compile(optimizer= 'adam', loss='binary_crossentropy', metrics=['accuracy'])
  return cnn

seed = 7
numpy.random.seed(seed)

train_datagen = ImageDataGenerator(rescale=1.0/255.0, 
                                   shear_range = 0.2,
                                   zoom_range = 0.2, 
                                   horizontal_flip = True )

#val_datagen = ImageDataGenerator(rescale=1.0/255.0) 
test_datagen = ImageDataGenerator(rescale=1.0/255.0) 

train_generator = train_datagen.flow_from_directory(directory = train_path, 
                                                    target_size = (hyper_dimension, hyper_dimension),
                                                    batch_size = hyper_batch_size, 
                                                    color_mode = hyper_mode,
                                                    class_mode = 'binary', 
                                                    seed = 42)

test_generator = test_datagen.flow_from_directory(directory = test_path, 
                                                 target_size = (hyper_dimension, hyper_dimension),
                                                 batch_size = hyper_batch_size, 
                                                 class_mode = 'binary',
                                                 color_mode = hyper_mode,
                                                 shuffle=False,
                                                 seed = 42)

test_generator.reset()

#datagen = ImageDataGenerator(rescale=1.0/255.0)
#dataset = datagen.flow_from_directory('dataset_dog_breeds/train/', class_mode='categorical')

X, Y = train_generator.next()

cnn = KerasClassifier(build_fn=create_model, verbose=0,batch_size=40, epochs=100)

#batch_size = [10, 20, 40, 60, 80, 100]
#epochs = [10, 50, 100]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
filters=[4,8,16,32,64,80,128]

'''param_grid = {
    'batch_size': batch_size, 
    'epochs':epochs,
    'optimizer': optimizer,
    'init_mode': init_mode,
    'activation': activation
}'''

param_grid= dict(filters=filters)
grid = GridSearchCV(estimator=cnn, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Found 1656 images belonging to 2 classes.
Found 415 images belonging to 2 classes.
Best: 0.727273 using {'filters': 8}
0.693939 (0.183377) with: {'filters': 4}
0.727273 (0.222681) with: {'filters': 8}
0.660606 (0.148701) with: {'filters': 16}
0.657576 (0.080060) with: {'filters': 32}
0.727273 (0.222681) with: {'filters': 64}
0.727273 (0.222681) with: {'filters': 80}
0.727273 (0.222681) with: {'filters': 128}


**For checking best kernel size**

In [21]:
def create_model(kernel_size=3):
  cnn=Sequential()
  cnn.add(InputLayer(input_shape=(hyper_dimension, hyper_dimension, hyper_channels)))

  cnn.add(Conv2D(filters=16, kernel_size=3, activation='relu'))
  cnn.add(MaxPooling2D(pool_size=(2,2)))

  cnn.add(Conv2D(filters=32, kernel_size=3, activation='relu'))
  cnn.add(MaxPooling2D(pool_size=(2,2)))

  cnn.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
  cnn.add(MaxPooling2D(pool_size=(2,2)))

  cnn.add(Flatten())

  cnn.add(Dense(activation='relu', units=128))
  cnn.add(Dense(activation='sigmoid', units=1))

  #cnn.summary()

  cnn.compile(optimizer= 'adam', loss='binary_crossentropy', metrics=['accuracy'])
  return cnn

seed = 7
numpy.random.seed(seed)

train_datagen = ImageDataGenerator(rescale=1.0/255.0, 
                                   shear_range = 0.2,
                                   zoom_range = 0.2, 
                                   horizontal_flip = True )

#val_datagen = ImageDataGenerator(rescale=1.0/255.0) 
test_datagen = ImageDataGenerator(rescale=1.0/255.0) 

train_generator = train_datagen.flow_from_directory(directory = train_path, 
                                                    target_size = (hyper_dimension, hyper_dimension),
                                                    batch_size = hyper_batch_size, 
                                                    color_mode = hyper_mode,
                                                    class_mode = 'binary', 
                                                    seed = 42)

test_generator = test_datagen.flow_from_directory(directory = test_path, 
                                                 target_size = (hyper_dimension, hyper_dimension),
                                                 batch_size = hyper_batch_size, 
                                                 class_mode = 'binary',
                                                 color_mode = hyper_mode,
                                                 shuffle=False,
                                                 seed = 42)

test_generator.reset()

#datagen = ImageDataGenerator(rescale=1.0/255.0)
#dataset = datagen.flow_from_directory('dataset_dog_breeds/train/', class_mode='categorical')

X, Y = train_generator.next()

cnn = KerasClassifier(build_fn=create_model, verbose=0,batch_size=40, epochs=100)

#batch_size = [10, 20, 40, 60, 80, 100]
#epochs = [10, 50, 100]
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
kernel_size=[2,3,5,7,9,11,13]

'''param_grid = {
    'batch_size': batch_size, 
    'epochs':epochs,
    'optimizer': optimizer,
    'init_mode': init_mode,
    'activation': activation
}'''

param_grid= dict(kernel_size=kernel_size)
grid = GridSearchCV(estimator=cnn, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Found 1656 images belonging to 2 classes.
Found 415 images belonging to 2 classes.
Best: 0.727273 using {'kernel_size': 9}
0.660606 (0.105322) with: {'kernel_size': 2}
0.721212 (0.124501) with: {'kernel_size': 3}
0.660606 (0.148701) with: {'kernel_size': 5}
0.657576 (0.080060) with: {'kernel_size': 7}
0.727273 (0.222681) with: {'kernel_size': 9}
0.596970 (0.104006) with: {'kernel_size': 11}
0.693939 (0.183377) with: {'kernel_size': 13}
